# Installation and Setup

In [ ]:
!pip install python-aiconfig

import openai

# Use your OpenAI Key

# Colab Secrets Example:
# from google.colab import userdata
# openai.api_key = userdata.get('openai_key')

# Environment Variable
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Load AI Config `travel.aiconfig.json`
Download file [from here](https://github.com/lastmile-ai/aiconfig/blob/main/cookbooks/Getting-Started/travel.aiconfig.json). Upload to 'Files' in your colab notebook and load (shown below).


In [30]:
from aiconfig import AIConfigRuntime, InferenceOptions, CallbackManager

# Load the aiconfig.
config = AIConfigRuntime.load('travel.aiconfig.json')
config.callback_manager = CallbackManager([])

# Run `get_activities` prompt with streaming




In [31]:
# Run a single prompt (with streaming). inference_options = InferenceOptions(stream=True)

inference_options = InferenceOptions(stream=True)
activities = await config.run("get_activities", options=inference_options)


1. Visit Central Park: Take a leisurely stroll, rent a bike, have a picnic or simply relax in the beautiful setting of Central Park.
2. Explore Times Square: Revel in the vibrant atmosphere, bright lights, and iconic billboards of Times Square, and catch a Broadway show if you can.
3. Walk the Brooklyn Bridge: Enjoy stunning views of the Manhattan skyline while walking across the famous Brooklyn Bridge.
4. Visit the Statue of Liberty: Take a ferry ride to Liberty Island and get up close to the iconic Statue of Liberty, a symbol of freedom and democracy.
5. Explore the Metropolitan Museum of Art: Discover one of the world's largest art museums, showcasing a vast collection of artwork spanning various periods and cultures.
6. Ride to the Top of the Empire State Building: Take in the breathtaking views of New York City's skyline from the observation decks of the iconic Empire State Building.
7. Experience the High Line: Walk along this elevated park built on an old railway track, offering

# Run `gen_itinerary` prompt


In [32]:
# This time, let's also add a callback manager to log more detailed events
config.callback_manager = CallbackManager([])

itinerary = await config.run(
    "gen_itinerary",
    params={"order_by": "duration"},
    run_with_dependencies=True,
    options=inference_options)

INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'gen_itinerary', 'params': {'order_by': 'duration'}, 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'kwargs': {'run_with_dependencies': True}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'get_activities', 'params': {'order_by': 'duration'}, 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'kwargs': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='get_activities', input='Tell me 10 fun attractions to do in NYC.', metadata=None, outputs=[ExecuteResult(output_type='execute_result', execution_count=0, data={'content': "1. Visit Central Park: Take a leisurely stroll, rent a bike, have a picnic or simply relax in the beautiful setting of C

1. Visit the iconic Times Square and immerse yourself in the vibrant atmosphere.
2. Take a stroll through Central Park, rent a rowboat, or enjoy a picnic in this beautiful urban oasis.
3. Explore the world-famous Metropolitan Museum of Art (the Met) and admire its extensive collection spanning various eras and cultures.
4. Ascend to the Top of the Rock Observation Deck at Rockefeller Center for panoramic views of the city.
5. Enjoy a Broadway show and experience the excitement of live theater.
6. Visit the 9/11 Memorial & Museum to pay tribute to the victims of the World Trade Center attacks and learn about their stories.
7. Roam around the charming neighborhoods of SoHo and Greenwich Village, known for their unique boutiques, cafes, and art galleries.
8. Take a ferry to Governors Island and enjoy biking, picnicking, or simply relaxing in this peaceful retreat from the city hustle.
9. Discover the fascinating exhibits at the American Museum of Natural History, including the dinosaur fo

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'content': '1. Visit the iconic Times Square and immerse yourself in the vibrant atmosphere.\n2. Take a stroll through Central Park, rent a rowboat, or enjoy a picnic in this beautiful urban oasis.\n3. Explore the world-famous Metropolitan Museum of Art (the Met) and admire its extensive collection spanning various eras and cultures.\n4. Ascend to the Top of the Rock Observation Deck at Rockefeller Center for panoramic views of the city.\n5. Enjoy a Broadway show and experience the excitement of live theater.\n6. Visit the 9/11 Memorial & Museum to pay tribute to the victims of the World Trade Center attacks and learn about their stories.\n7. Roam around the charming neighborhoods of SoHo and Greenwich Village, known for their unique boutiques, cafes, and art galleries.\n8. Take a ferry to Governors I

1. Start your day by visiting the iconic Times Square and immerse yourself in the vibrant atmosphere (Estimated Duration: 1.5 Hours - 8:00 AM to 9:30 AM)
2. Take a stroll through Central Park, rent a rowboat, or enjoy a picnic in this beautiful urban oasis (Estimated Duration: 2.5 Hours - 10:00 AM to 12:30 PM)
3. Have lunch while exploring the diverse food scene of NYC, from trendy food halls to iconic pizza joints and Michelin-starred restaurants (Estimated Duration: 2 Hours - 1:00 PM to 3:00 PM)
4. Spend your early afternoon at the Metropolitan Museum of Art (the Met). Marvel at the extensive collection spanning various eras and cultures (Estimated Duration: 3 Hours - 3:30 PM to 6:30 PM)
5. Next, roam around the charming neighborhoods of SoHo and Greenwich Village, known for their unique boutiques, cafes, and art galleries (Estimated Duration: 2 Hours - 7:00 PM to 9:00 PM)
6. Begin your second day by visiting the 9/11 Memorial & Museum to pay tribute to the victims of the World Trade

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'content': '1. Start your day by visiting the iconic Times Square and immerse yourself in the vibrant atmosphere (Estimated Duration: 1.5 Hours - 8:00 AM to 9:30 AM)\n2. Take a stroll through Central Park, rent a rowboat, or enjoy a picnic in this beautiful urban oasis (Estimated Duration: 2.5 Hours - 10:00 AM to 12:30 PM)\n3. Have lunch while exploring the diverse food scene of NYC, from trendy food halls to iconic pizza joints and Michelin-starred restaurants (Estimated Duration: 2 Hours - 1:00 PM to 3:00 PM)\n4. Spend your early afternoon at the Metropolitan Museum of Art (the Met). Marvel at the extensive collection spanning various eras and cultures (Estimated Duration: 3 Hours - 3:30 PM to 6:30 PM)\n5. Next, roam around the charming neighborhoods of SoHo and Greenwich Village, known for their un

# Add prompt to config

In [39]:
# Add gen_packing_list as a prompt to config
from aiconfig import Prompt
model = "gpt-3.5-turbo"
data = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You provide a bulleted list of items to pack for a week long trip."
        },
        {
            "role": "user",
            "content": "What should I bring to {{location}}?"
        }
    ]
}

new_prompts = await config.serialize(model, data, prompt_name="gen_packing_list", params={"location": "nyc"})
config.add_prompt(prompt_name="gen_packing_list", prompt_data=new_prompts[0])

INFO:my-logger:Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}, {'role': 'user', 'content': 'What should I bring to {{location}}?'}]}, 'prompt_name': 'gen_packing_list', 'params': {'location': 'nyc'}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'gen_packing_list', 'data': {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}, {'role': 'user', 'content': 'What should I bring to {{location}}?'}]}, 'parameters': {'location': 'nyc'}, 'kwargs': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' data={'res

In [40]:
packing_list = await config.run("gen_packing_list", params=None, options=inference_options)

INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'gen_packing_list', 'params': None, 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'kwargs': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_packing_list', input='What should I bring to {{location}}?', metadata=PromptMetadata(model=ModelMetadata(name='gpt-3.5-turbo', settings={'system_prompt': {'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}}), tags=None, parameters={'location': 'nyc'}, remember_chat_context=True), outputs=[]), 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'parameters': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_packin

Here's a list of items you should consider packing for your trip to NYC:

1. Comfortable walking shoes: NYC is a city best explored on foot, so be sure to pack comfortable shoes for all the walking you'll be doing.
2. Lightweight clothing: Depending on the season, pack layered clothing that can accommodate fluctuating temperatures. Summers can be hot and humid while winters can be cold and snowy.
3. Weather-appropriate outerwear: Bring a waterproof jacket or umbrella, especially if you're visiting during the rainy season. In colder months, pack a warm coat, hat, and gloves.
4. Daypack or tote bag: Carry a small bag to hold your personal items, snacks, water bottle, and any souvenirs you might pick up during the day.
5. Portable phone charger: With all the sightseeing and navigating you'll be doing, it's important to ensure your phone stays charged throughout the day.
6. Travel guide or map: Even with smartphones, having a physical map or guidebook can be handy for quick reference or if

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'content': "Here's a list of items you should consider packing for your trip to NYC:\n\n1. Comfortable walking shoes: NYC is a city best explored on foot, so be sure to pack comfortable shoes for all the walking you'll be doing.\n2. Lightweight clothing: Depending on the season, pack layered clothing that can accommodate fluctuating temperatures. Summers can be hot and humid while winters can be cold and snowy.\n3. Weather-appropriate outerwear: Bring a waterproof jacket or umbrella, especially if you're visiting during the rainy season. In colder months, pack a warm coat, hat, and gloves.\n4. Daypack or tote bag: Carry a small bag to hold your personal items, snacks, water bottle, and any souvenirs you might pick up during the day.\n5. Portable phone charger: With all the sightseeing and navigating y

# 5. Save the AIConfig with outputs


In [37]:
# Save the aiconfig to disk. and serialize outputs from the model run
config.save('updated.aiconfig.json', include_outputs=True)

# 6. Open the AIConfig in AI Workbook Playground
1. Download `updated.aiconfig.json`.
2. Go to https://lastmileai.dev.
3. Go to Workbooks page: https://lastmileai.dev/workbooks.
4. Click dropdown '+New Workbook' and select 'Create from AIConfig'
5. Upload `updated.aiconfig.json`.

Try out the workbook playground here: [NYC Travel Workbook](https://lastmileai.dev/workbooks/clooqs3p200kkpe53u6n2rhr9)